In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.inspection import PartialDependenceDisplay
from sklearn.ensemble import GradientBoostingRegressor

import matplotlib.pyplot as plt
%matplotlib inline


Partial dependency plots
==========================

Partial dependency plots consists in evaluating the model prediction
while varying one of more features and marginalizing (averaging) out the
effect of the other features.

They can be computed by varying only one feature, or more



In [ ]:
# Load and prepare the data
X, y = datasets.fetch_california_housing(
    as_frame=True, return_X_y=True
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1
)
X.head()

Train a GBRT



In [ ]:
clf = GradientBoostingRegressor(
    n_estimators=100, max_depth=4, learning_rate=0.1,
    loss='huber', random_state=1
)
clf.fit(X_train, y_train)

All-in-one plotting function
-----------------------------



Inspect feature 0, 3, 5, 6, and the interaction between 5 and 0, and 5
and 3



In [ ]:
target_features = [
    'MedInc', 'AveOccup', 'HouseAge', 'AveRooms',
    ('AveOccup', 'HouseAge'), ('Latitude', 'Longitude')
]
fig, ax = plt.subplots(figsize=(15, 10))
PartialDependenceDisplay.from_estimator(
    clf, X_test, target_features,
    feature_names=X.columns,
    grid_resolution=50, ax=ax
)
plt.tight_layout()

Lower-level partial_dependence function
----------------------------------------



In [ ]:
target_feature = ('Latitude', 'Longitude')
import numpy as np
from sklearn.inspection import partial_dependence


pdp = partial_dependence(
    clf, X=X_test, features=target_feature,
    grid_resolution=50, kind='average'
)

# The 2D coordinate grid (for plotting)
XX, YY = np.meshgrid(pdp['values'][0], pdp['values'][1])

# Reshape the partial deps on the grid
Z = pdp.average[0].T

3D plotting



In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = Axes3D(fig, auto_add_to_figure=False)
fig.add_axes(ax)
surf = ax.plot_surface(XX, YY, Z, rstride=1, cstride=1,
                       cmap=plt.cm.BuPu, edgecolor='k')
ax.set_xlabel(target_feature[0])
ax.set_ylabel(target_feature[1])
ax.set_zlabel('Partial dependence')
#  pretty init view
ax.view_init(elev=28, azim=70)
plt.colorbar(surf)